### Section with regressions for individual students and institutions

In this section we explore the results of section 5.1 - the static factors in order to establish relationship between socio-economic factors and results. Furthermore, we introduce some group based factors such as class size

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sas7bdat
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.preprocessing import MinMaxScaler
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [ ]:
import matplotlib as mpl

mpl.style.use('seaborn-whitegrid')  # Use 'seaborn-whitegrid' instead of 'ggplot'

mpl.rcParams['font.family'] = 'serif'
mpl.rcParams['font.size'] = 14
mpl.rcParams['axes.labelsize'] = 13
mpl.rcParams['axes.titlesize'] = 16
mpl.rcParams['xtick.labelsize'] = 11
mpl.rcParams['ytick.labelsize'] = 11
mpl.rcParams['axes.titleweight'] = 'bold'
mpl.rcParams['axes.spines.top'] = False
mpl.rcParams['axes.spines.right'] = False
mpl.rcParams['axes.prop_cycle'] = plt.cycler('color', plt.cm.Set1.colors)

plt.rcParams['figure.figsize'] = [10, 6]
plt.rcParams['figure.dpi'] = 200
plt.rcParams['savefig.dpi'] = 300
plt.rcParams['figure.facecolor'] = '#f8f8f8'

pd.set_option('display.max_columns', None)

In [ ]:
#Base knowledge of students in a given period
elev=pd.read_pickle('../data/clean_reg.pkl')
elev.rename(columns={'INSTNR':'instnr'},inplace=True)
#df=df[['elev_id','startdato','slutdato']]

In [ ]:
#Read Gender data
filepath='../data/macom_køn.sas7bdat'
gender =pd.read_sas(filepath, format='sas7bdat',encoding='iso-8859-1')

gender.rename(columns={'PERSON_ID':'elev_id'},inplace=True)

#Merge gender and student data 
elev=elev.merge(gender,on='elev_id',how='left')

elev['FOED_DATO']=pd.to_datetime(elev['FOED_DATO'])

#Find the students age at the start of the period
elev['age_semester']=elev['startdato']-elev['FOED_DATO']

In [ ]:
#Both absence, lectures and percentage absence. 
lec=pd.read_pickle('../df/lecture_absence.pkl')

In [ ]:
#Grades for students seperated on periods
grade=pd.read_pickle('figures/grades_steps.pkl')
grade.drop(columns=['step'],inplace=True)

In [ ]:
#Grades for students seperated on periods
grade=pd.read_pickle('figures/grades_steps.pkl')
grade.drop(columns=['step'],inplace=True)

In [ ]:
#Transformed socio_economic features, with highest education level
socio=pd.read_pickle('../df/edu_parents.pkl')

In [ ]:
#Do the necessary merging, I add students to absence and enrich the lectures with grades and student info
df = lec.merge(elev,on=['elev_id','startdato','slutdato','step']).merge(grade, on=['elev_id','startdato','slutdato'])

In [ ]:
#Add the socio factors 
test=df.merge(socio,on='elev_id',how='left')
test.dropna(inplace=True)#Drop na 
test['percent']=test['percentage']*100 #Calculate percentage as a whole number 

In [ ]:
#Get the metadata for each institution to undestand difference between students and their educational institution. 
insti=(test.groupby(['instnr'],sort=False)
              .agg(**{'grade_avg': ('avg_grade','mean'),'school_absence':('percent','mean')})
              .reset_index()
              )

#Add institutional data to the students 
temp=test.merge(insti,on='instnr')


#Calc difference in students to their school. 
temp['off_absence']=temp['percent'] - temp['school_absence']
temp['off_grade']=temp['avg_grade']- temp['grade_avg']

#Drop columns not used for now
pd.set_option('display.max_columns',None)
temp.drop(columns=['depart_reason','join_reason','grade_avg','school_absence'],inplace=True)

In [ ]:
#Join the average group size to the students
groups_size=pd.read_pickle('../df/groups_size.pkl')
#test=temp.merge(groups_size,on=['elev_id','startdato','slutdato'],how='left')
test=temp.merge(groups_size,on=['elev_id','startdato','slutdato','step','inst_nr'],how='left')
test=test.dropna()#We only miss 2000 students from adding the group size

In [ ]:
teacher_comp=pd.read_pickle('../df/student_teacher_comp.pkl')
teacher_comp.drop(columns=['max_sum','other'],inplace=True)

In [ ]:
new_test=test.merge(teacher_comp,on=['elev_id','startdato','slutdato'],how='left')

In [ ]:
new_test=new_test[~new_test['pedagogical_percent'].isna()]

In [ ]:
import math
from scipy.stats import zscore 
#remove outliers for income for first father, then mother 
mean=new_test['income_father'].mean()
std=new_test['income_father'].std()

treshold=2.5
outlier=new_test[(new_test['income_father'] - mean).abs() > treshold * std]

new_test=new_test.drop(outlier.index)

#remove outliers for income for first father, then mother 
mean1=new_test['income_mother'].mean()
std1=new_test['income_mother'].std()

outlier1=new_test[(new_test['income_mother'] - mean1).abs() > treshold * std1]

new_test=new_test.drop(outlier1.index)

#Change all values less than 0 to the lowest values 0.1 

new_test[['income_father','income_mother']]=new_test[['income_father','income_mother']].applymap(lambda x: 0.1 if x <= 0 else x)
new_test['income_mother']=np.log(new_test['income_mother'])
new_test['income_father']=np.log(new_test['income_father'])

#Convert education level 5 to education level 6, because they are similar. 
new_test['edu_level_mother']=new_test['edu_level_mother'].astype(int)
new_test['edu_level_father']=new_test['edu_level_father'].astype(int)

#I have other absence columns and group size columns
new_test.drop(columns=['percentage','absence'],inplace=True)
new_test.rename(columns={'students':'group_size'},inplace=True)

#Select columns to normalize
cols_to_normalize=['income_mother','income_father','edu_level_father','edu_level_mother','pedagogical_percent',
                    'group_size']

#Normalize columns with z-score
new_test[cols_to_normalize]=new_test[cols_to_normalize].apply(zscore)

df=new_test.copy()

In [ ]:
#Ensure string
df['social_mother']=df['social_mother'].astype(str)
df['social_father']=df['social_father'].astype(str)

#Convert the social_status to a booelean
df['social_father'] = df['social_father'].map({'outside_of_worforce': False, 'employed': True})

df['social_mother'] = df['social_mother'].map({'outside_of_worforce': False, 'employed': True})

In [ ]:
#Rename chosen columns
df.rename(columns={'KOEN':'gender'},inplace=True)
df.drop(columns=['FOED_DATO'],inplace=True)

men=df[df['gender']==1]
women=df[df['gender']==2]

In [ ]:
#Create a dataframe for each step
df1=df[df['step']==1]
df2=df[df['step']==2]
df3=df[df['step']==3]

In [ ]:
#Normalize start age for year 3 students
df3['age_semester']=df3['age_semester'].dt.total_seconds().round().astype(int)

cols_to_normalize=['age_semester']


df3[cols_to_normalize]=df3[cols_to_normalize].apply(zscore)


df3_men=df3[df3['gender']==1]
df3_wom=df3[df3['gender']==2]

#### Machine learning model to calculate social strength score to optimize grade prediction 

In [ ]:
from sklearn.linear_model import Lasso, Ridge
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

X = df3[['social_father','social_mother','income_father','income_mother','edu_level_mother','edu_level_father']]

y = df3['avg_grade']

X_train,X_val,y_train,y_val = train_test_split(X,y,test_size=0.2,random_state=42)

#Train the Lasso or Rigde regression model
alpha=0.1 #Adjust based on the cross-validation
model = Lasso(alpha=alpha)
model.fit(X_train,y_train)

y_pred=model.predict(X_val)
mse=mean_squared_error(y_val,y_pred)
print("Mean squared error", mse)
weights = model.coef_
feature_names = X.columns

print("Optimal weights:")
for i in range(len(weights)):
    print(feature_names[i], ":", weights[i])
    
baseline_pred = [y_train.mean()] * len(y_val)
baseline_mse = mean_squared_error(y_val, baseline_pred)
print("\n______________-------------------__________________\nBaseline mean squared error:", baseline_mse)

#### Regression for individual students

In [ ]:
# Create a copy of the dataframe
df4 = df3.copy()

# Replace True with 0 and False with 1 in social_father and social_mother variables
df4['social_father'] = df4['social_father'].replace({True: 0, False: 1})
df4['social_mother'] = df4['social_mother'].replace({True: 0, False: 1})

# Run the regression with 1 for False and 0 for True
mod = smf.ols(formula='avg_grade ~ C(social_father) + C(social_mother) + edu_level_father + edu_level_mother + ' +
            ' income_father + income_mother + group_size +' + 
            ' pedagogical_percent + age_semester + C(gender)', data=df4)
res = mod.fit()

# Print the summary of the model
print(res.summary())

In [ ]:
#Run with gender as interaction terms
mod_all = smf.ols(formula='avg_grade ~ income_mother + income_father + edu_level_father + edu_level_mother+ C(social_father) +' + 
            'C(social_mother) + age_semester + age_semester*gender + income_mother*gender + income_father*gender + edu_level_father*gender + edu_level_mother*gender' + 
                  '+C(social_father)*gender + C(social_mother)*gender'
            ,data=df4)

res_all = mod_all.fit()
print(res_all.summary())


In [ ]:
# Define x-tick labels
labels = ['Father unemployed','Mother unemployed','Mother income', 'Father income', 'Father edu level', 'Mother edu level', 'Age semester']

df4_men=df4[df4['gender']==1]
df4_wom=df4[df4['gender']==2]

# All
mod_all = smf.ols(formula='avg_grade ~ C(social_father) + C(social_mother) +  ' +
            ' income_father + income_mother + edu_level_father + edu_level_mother + age_semester', data=df4)
res_all = mod_all.fit()

# Women
mod_women = smf.ols(formula='avg_grade ~ C(social_father) + C(social_mother) +' +
            ' income_father + income_mother + edu_level_father + edu_level_mother + age_semester'
            ,data=df4_wom)
res_women = mod_women.fit()

# Men
mod_men = smf.ols(formula='avg_grade ~ + C(social_father) + C(social_mother) + ' +
            ' income_father + income_mother + edu_level_father + edu_level_mother + age_semester'
            ,data=df4_men)
res_men = mod_men.fit()

#Extract coefficient estimates and standard errors
coef_all = res_all.params
se_all = res_all.bse
coef_women = res_women.params
se_women = res_women.bse
coef_men = res_men.params
se_men = res_men.bse

#Combine coefficient estimates and standard errors into a single dataframe
coef_df = pd.concat([coef_all,coef_women, coef_men], axis=1)
coef_df.columns = ['All','Female', 'Male']
coef_df['All SE'] = se_all
coef_df['Women SE'] = se_women
coef_df['Men SE'] = se_men

# Drop the "Intercept" row from the DataFrame
coef_df = coef_df.drop(index='Intercept')

# Create bar chart of coefficient estimates with error bars
ax = coef_df[['All','Female', 'Male']].plot(kind='bar', yerr=coef_df[['All SE','Women SE', 'Men SE']].T.values, 
                                          capsize=5, figsize=(10, 6))
ax.set_xticklabels(labels, rotation=45, ha='right')
plt.ylabel('Coefficient Estimate')
plt.title('Comparison of Coefficient Estimates for Grades between Women and Men')
plt.savefig('figures/coefficient_gender.pdf')
plt.show()


Working on showing the cross-correlation matrix for the individual dataframe 

In [ ]:
#Create df with only regression columns
tmp=df3[['income_mother','income_father','edu_level_father','edu_level_mother','gender',
                    'group_size','age_semester','percent','avg_grade']]

cols=['Income Mother','Income Father','Education Father','Education Mother','Gender',
                    'Class size','Age','Absence','Grade Average']

#Transpose the data 
corr_matrix=tmp.corr()

#Create heatmap
fig,ax=plt.subplots(figsize=(16,12))
im=ax.imshow(corr_matrix,cmap='coolwarm',origin='lower')

ax.set_xticks(np.arange(len(cols)))
ax.set_yticks(np.arange(len(cols)))
ax.set_xticklabels(cols)
ax.set_yticklabels(cols)

#Rotate the tick labels and set their alignments
plt.setp(ax.get_xticklabels(),rotation=45,ha='right',rotation_mode='anchor')
                   
#Get values in the boxes
for i in range(len(cols)):
    for j in range(len(cols)):
        text=ax.text(j,i,round(corr_matrix.iloc[i,j],2),
                    ha='center',va='center',color='w')

cbar=ax.figure.colorbar(im,ax=ax)
cbar.ax.set_ylabel("Correlation",rotation=-90,va='bottom')
ax.set_title("Cross-correlation matrix")
ax.grid(False)
plt.savefig('figures/cross_corelation_matrix.pdf')
tmp.to_pickle('figures/cross_corelation_matrix.pkl')


plt.show()

### Do regression for instiuttions

In [ ]:
#Get the metadata for each institution to undestand difference between students and their educational institution. 
tmp=df3.copy()

tmp['gender'] = tmp['gender'].replace({1:0, 2:1})

inst=(tmp.groupby('instnr',sort=False)
          .agg(**{'avg_grade': ('avg_grade','mean'),'absence_percent':('percent','mean'),'income_mother':('income_mother','mean'),
                 'income_father':('income_father','mean'),'pedagogical_percent':('pedagogical_percent','mean'),
                'edu_level_father':('edu_level_father','mean'),'edu_level_mother':('edu_level_mother','mean'),
                 'group_size':('group_size','mean'),'social_father_pct': ('social_father', 'mean'),
                 'social_mother_pct': ('social_mother', 'mean'),'average_age': ('age_semester', 'mean'),
                 'pct_female':('gender','mean')})
          .reset_index()
)

In [ ]:
#Normalize start age for year 3 students
cols_to_normalize=['social_mother_pct','social_father_pct','pct_female']


inst[cols_to_normalize]=inst[cols_to_normalize].apply(zscore)

In [ ]:
#Create df with only regression columns
tmp=inst[['income_mother','income_father','social_father_pct', 'social_mother_pct','edu_level_father',
          'edu_level_mother','pedagogical_percent','group_size','avg_grade','absence_percent']]

cols=['Income Mother','Income Father','Employment Father','Employment Mother','Education Level Father',
      'Education level Mother','Pedagogical Percentage','Group Size','Average Grade','Absence Percentage']

#Transpose the data 
corr_matrix=tmp.corr()

#Create heatmap
fig,ax=plt.subplots(figsize=(16,12))
im=ax.imshow(corr_matrix,cmap='coolwarm',origin='lower')

ax.set_xticks(np.arange(len(cols)))
ax.set_yticks(np.arange(len(cols)))
ax.set_xticklabels(cols)
ax.set_yticklabels(cols)

#Rotate the tick labels and set their alignments
plt.setp(ax.get_xticklabels(),rotation=45,ha='right',rotation_mode='anchor')
                   
#Get values in the boxes
for i in range(len(cols)):
    for j in range(len(cols)):
        text=ax.text(j,i,round(corr_matrix.iloc[i,j],2),
                    ha='center',va='center',color='w')

cbar=ax.figure.colorbar(im,ax=ax)
cbar.ax.set_ylabel("Correlation",rotation=-90,va='bottom')
ax.set_title("Cross-correlation matrix")
ax.grid(False)
plt.savefig('figures/cross_corelation_matrix_inst.pdf')
tmp.to_pickle('figures/cross_corelation_matrix_inst.pkl')


plt.show()

In [ ]:
#Regression results for institutions
mod=smf.ols(formula='avg_grade ~  edu_level_father + edu_level_mother + income_father + income_mother ' 
         
            ,data=inst)

res=mod.fit()

print(res.summary())

In [ ]:
#Regression results absence institutions
mod=smf.ols(formula='absence_percent ~ edu_level_father + edu_level_mother +income_father + ' +
            ' income_mother + pedagogical_percent'
            ,data=inst)

res=mod.fit()

print(res.summary())

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score
from sklearn.inspection import PartialDependenceDisplay


#Experiment with RFR to see if the scores are better
# Define the features (independent variables) and the target variable
features = ['edu_level_father', 'edu_level_mother', 'income_father', 'income_mother']
target = 'avg_grade'

# Split the data into features and target variable
X = inst[features]
y = inst[target]

# Create a Random Forest Regressor model
rf_model = RandomForestRegressor()

# Fit the model to the data
rf_model.fit(X, y)

# Predict the target variable using the trained model
y_pred = rf_model.predict(X)

# Calculate the R-squared score
r2 = r2_score(y, y_pred)
# Format the R-squared score as a string
r2_str = f"{r2:.4f}"


# Get the feature importances from the trained model
feature_importances = rf_model.feature_importances_

# Print the feature importances
for feature, importance in zip(features, feature_importances):
    print(f"{feature}: {importance}")

    
n_cols=2
n_rows=int((len(X.columns)/n_cols))

fig, ax = plt.subplots(n_rows,n_cols,figsize=(12,10))

# Create PartialDependenceDisplay object
display = PartialDependenceDisplay.from_estimator(rf_model, X, features,ax=ax,n_cols=n_cols)

fig.suptitle(f"Partial Dependence Plot, R2 = {r2_str}")

fig.tight_layout();

In [ ]:
from sklearn.preprocessing import StandardScaler

#Grade
temp=inst.copy()
temp['absence_percent']=temp['absence_percent']*-1

# standardize the variables
scaler = StandardScaler()
temp[['edu_level_father', 'edu_level_mother', 'income_father', 'income_mother', 'pedagogical_percent', 'avg_grade', 'absence_percent']] = scaler.fit_transform(temp[['edu_level_father', 'edu_level_mother', 'income_father', 'income_mother', 'pedagogical_percent', 'avg_grade', 'absence_percent']])

inst_grade = smf.ols(formula='avg_grade ~ edu_level_father + edu_level_mother' +
            ' + income_father + income_mother'
            ,data=temp)
res_grade = inst_grade.fit()

#Absence
inst_absence = smf.ols(formula='absence_percent ~  edu_level_father + edu_level_mother' +
            ' + income_father + income_mother + pedagogical_percent'
            ,data=temp)
res_absence = inst_absence.fit()

# Extract coefficient estimates and standard errors
coef_grade = res_grade.params
se_grade = res_grade.bse
coef_absence = res_absence.params
se_absence = res_absence.bse


# Combine coefficient estimates and standard errors into a single dataframe
coef_df = pd.concat([coef_grade,coef_absence], axis=1)
coef_df.columns = ['Grade','Absence']
coef_df['Grade SE'] = se_grade
coef_df['Absence SE'] = se_absence

# Drop the "intercept" row from the DataFrame
coef_df = coef_df.drop(index='Intercept')

# Create bar chart of coefficient estimates with error bars
ax = coef_df[['Grade','Absence']].plot(kind='bar', yerr=coef_df[['Grade SE','Absence SE']].T.values, 
                                       capsize=5, figsize=(10, 6))
ax.set_xticklabels(['Education Level Father', 'Education Level Mother', 'Income Father', 'Income Mother', 'Pedagogical Percentage'], rotation=45)
plt.ylabel('Coefficient Estimate Standard-Scaled')
plt.title('Comparison of Coefficient Estimates for Institutions Between Grades and Absence')
plt.savefig('figures/coef_inst.pdf')
plt.show()


In [ ]:
#test to see variable importance for institutions
X = inst[['social_father_pct','social_mother_pct','income_father','income_mother','edu_level_mother','edu_level_father']]

y = inst['avg_grade']

X_train,X_val,y_train,y_val = train_test_split(X,y,test_size=0.2,random_state=42)

#Train the Lasso or Rigde regression model
alpha=0.1 #Adjust based on the cross-validation
model = Lasso(alpha=alpha)
model.fit(X_train,y_train)

y_pred=model.predict(X_val)
mse=mean_squared_error(y_val,y_pred)
print("Mean squared error", mse)
weights = model.coef_
feature_names = X.columns

print("Optimal weights:")
for i in range(len(weights)):
    print(feature_names[i], ":", weights[i])

In [ ]:
#Plot histogram of different variables used as independent 
fig, axes = plt.subplots(nrows=2, ncols=3, figsize=(10,6))
for i, col in enumerate(X.columns):
    ax = axes[i // 3, i % 3]
    ax.hist(X[col],color='steelblue')
    ax.set_title(col,fontsize=12)
    
fig.suptitle('Histogram of socio-economic factors on school level', fontsize=16,fontweight='bold')

plt.tight_layout()
plt.savefig('figures/histogram_socio_institution.pdf')

In [ ]:
df4 = pd.read_pickle('../df/education_parents_age_student.pkl')
df4['highest_edu_level'] = df4[['edu_level_father', 'edu_level_mother']].max(axis=1)

# Create a boxplot of age semester by education level
ax = sns.boxplot(x='highest_edu_level', y='age_semester', color='steelblue', data=df4)

# Calculate the mean values for each education level
mean_values = df4.groupby('highest_edu_level')['age_semester'].mean()

# Add mean values as text annotations
for i, level in enumerate(mean_values.index):
    ax.text(i, mean_values[level], f'{mean_values[level]:.2f}', ha='center', va='top')

# Add labels and title
plt.xlabel('Highest Completed Education of Either Parent')
plt.ylabel('Age at Year 3')
plt.title('Relationship between Parents Highest Completed Education and Age of Student')

plt.savefig('figures/education_level_parents_age_student.pdf')
plt.show()
